In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

In [0]:


%run /Workspace/Consolidated_Pipeline/1_setup/utilities

In [0]:
print(bronze_schema, silver_schema, gold_schema)

In [0]:
dbutils.widgets.text("catalog", "fmcg", "catalog")
dbutils.widgets.text("data_source", "customer", "Data Source")

In [0]:
catalog = dbutils.widgets.get("catalog")
data_source = dbutils.widgets.get("data_source")

base_path = f"s3://sd-database/full_load/{data_source}/*.csv"


In [0]:
df = (
    spark.read.format("csv")
        .option("header", True)
        .option("inferSchema", True)
        .load(base_path)
        .withColumn("read_timestamp", F.current_timestamp())
        .select("*", "_metadata.file_name", "_metadata.file_size")
)
display(df.limit(10))

In [0]:
df.printSchema()

In [0]:
df.write\
    .format("delta")\
    .option("delta.enableChangeDataFeed", "true")\
    .mode("overwrite")\
    .saveAsTable(f"{catalog}.{bronze_schema}.{data_source}")

### Silver Processing

In [0]:
df_bronze = spark.sql(f"SELECT * FROM {catalog}.{bronze_schema}.{data_source};")
df_bronze.show(10)

In [0]:
df_bronze.printSchema()

In [0]:
df_duplicates = df_bronze.groupBy("customer_id").count().filter(F.col('count') > 1)
display(df_duplicates)

In [0]:
print('Rows before duplicates dropped: ', df_bronze.count())
df_silver = df_bronze.dropDuplicates(["customer_id"])
print('Rows after duplicates dropped: ', df_silver.count())

In [0]:
#check valuse that have leading and trailing spaces

display(
    df_silver.filter(F.col('customer_name') != F.trim(F.col("customer_name")))
)

In [0]:
df_silver = df_silver.withColumn(
    "customer_name",
    F.trim(F.col("customer_name"))
)

In [0]:
# ReCheck if there is space again
display(
    df_silver.filter(F.col('customer_name') != F.trim(F.col('customer_name')))
)

In [0]:
df_silver.select('city').distinct().show()

In [0]:
#  Typos != correct name

city_mapping = {
    'Bengaluruu': 'Bengaluru',
    'Bengalore': 'Bengaluru',

    'Hyderabadd': 'Hyderabad',
    'Hyderbad': 'Hyderabad',

    'NewDelhi': 'New Delhi',
    'NewDheli': 'New Delhi',
    'NewDelhee': 'New Delhi'
}

allowed = ['Bengaluru', 'Hyderabad', 'New Delhi']

df_silver = (
    df_silver
    .replace(city_mapping, subset=["city"])
    .withColumn(
        'city',
        F.when(F.col('city').isNull(), None)
        .when(F.col('city').isin(allowed), F.col('city'))
        .otherwise(None)
    )
)

df_silver.select('city').distinct().show()

In [0]:
df_silver.select('customer_name').distinct().show()

In [0]:
# title case fix
df_silver = df_silver.withColumn(
    'customer_name',
    F.when(F.col('customer_name').isNull(), None)
    .otherwise(F.initcap('customer_name'))
)

In [0]:
df_silver.select('customer_name').distinct().show()

### .5: Handling Missing Cities

In [0]:
df_silver.filter(F.col('city').isNull()).show(truncate=False)

In [0]:
null_city_customer_names = ['Sprintx Nutrition', 'Zenathlete Foods', 'Primefuel Nutrition', 'Recovery Lane']

df_silver.filter(F.col('customer_name').isin(null_city_customer_names)).show(truncate=False)

In [0]:
#  Business Confirmation Note: City corrections confirmed by business team

customer_city_fix = {
    # 'Sprintx Nutrition
    789403: "New Delhi",
    # 'Zenathlete Foods
    789420: "Bengaluru",
    # 'Primefuel Nutrition
    789521: 'Hyderabad',
    # 'Recovery Lane
    789603: 'Hyderabad',
    # 'Endurance Foods
    789101: 'Bengaluru',
    # primefuel nutrition
    789520: 'Bengaluru'
}

df_fix = spark.createDataFrame(
    [(cust_id,country) for cust_id, country in customer_city_fix.items()], 
    ['customer_id', 'fixed_city']
)

display(df_fix)

In [0]:
df_silver = (
    df_silver
    .join(df_fix, "customer_id", "left")
    .withColumn(
        'city',
        F.coalesce('city', 'fixed_city')
    )
    .drop('fixed_city')
)

display(df_silver)

In [0]:
null_city_customer_names = ['Sprintx Nutrition', 'Zenathlete Foods', 'Primefuel Nutrition', 'Recovery Lane']

df_silver.filter(F.col('customer_name').isin(null_city_customer_names)).show(truncate=False)

In [0]:
df_silver = df_silver.withColumn('customer_id', F.col('customer_id').cast('string'))

print(df_silver.printSchema())

In [0]:
df_silver = (
    df_silver
    .withColumn(
        'customer',
        F.concat_ws('-', 'customer_name', F.coalesce(F.col('city'), F.lit('Unknown')))
    )
    # Static attribute aligned with parent data model
    .withColumn('market', F.lit('India'))
    .withColumn('platform', F.lit('Sports Bar'))
    .withColumn('channel', F.lit('Acquisition'))
)

display(df_silver.limit(5))

In [0]:
df_silver.write\
    .format('delta') \
    .option('delta.enableChangeDataFeed', 'true') \
    .option('mergeSchema', 'true') \
    .mode('overwrite') \
    .saveAsTable(f'{catalog}.{silver_schema}.{data_source}')

### Gold Proecessing

In [0]:
df_silver = spark.sql(f'SELECT * FROM {catalog}.{silver_schema}.{data_source};') 
                      
#take required columns only

df_gold = df_silver.select('customer_id','customer_name','city','customer','market','platform','channel')

display(df_gold.show())

In [0]:
df_gold.write\
    .format('delta') \
    .option('delta.enableChangeDataFeed', 'true') \
    .mode('overwrite') \
    .saveAsTable(f'{catalog}.{gold_schema}.sb_dim_{data_source}')

In [0]:
from delta.tables import DeltaTable
import pyspark.sql.functions as F

delta_table = DeltaTable.forName(spark, 'fmcg.gold.dim_customers')

df_child_customers = spark.table('fmcg.gold.sb_dim_customers').select(
      F.col('customer_id').alias('customer_code'),
      'customer',
      'market',
      'platform',
      'channel'
)

display(df_child_customers.limit(10))

In [0]:
delta_table.alias('target').merge(
    source=df_child_customers.alias('source'),
    condition='target.customer_code = source.customer_code'
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()